In [131]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder,StandardScaler

In [132]:
df=pd.read_csv("data/stroke-data.csv")
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [133]:
#droping id column , unwanted column
df.drop('id',axis=1,inplace=True)
df.columns

Index(['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [134]:
imputer = SimpleImputer(strategy = 'mean')
df['bmi']=imputer.fit_transform(df[['bmi']])
#encoded_data= df.copy()
#2nd Scale down the numerical features
#features_to_scale=['age','bmi']
#scaler = MinMaxScaler()
#encoded_data[features_to_scale]=scaler.fit_transform(encoded_data[features_to_scale])

In [135]:
df.isna().sum()

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [136]:
#as the 'avg glucose level dosent have a normal distribution 
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# Initialize QuantileTransformer
#scaler = QuantileTransformer(output_distribution='uniform')

# Apply quantile transformation to avg_glucose_level
#encoded_data['avg_glucose_level'] = scaler.fit_transform(encoded_data[['avg_glucose_level']])

In [137]:
#df1 = encoded_data.copy()
#df1.columns
df.columns

Index(['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [138]:
X = df.drop('stroke',axis=1)
y = df['stroke']

In [141]:
df.dtypes

gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object

In [142]:
#create column transformer with 3 types of transformers
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler,LabelEncoder
from sklearn.compose import ColumnTransformer

# Initialize LabelEncoder for each categorical feature
label_encoders = {}
for col in cat_features:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

# Standardize the numerical features
scaler = StandardScaler()
X[num_features] = scaler.fit_transform(X[num_features])

# Now X is ready for modeling
print(X.head())
# Train the model
#pipeline.fit(X, y)
# Save the pipeline
#joblib.dump(pipeline, 'stroke_pipeline.pkl')

   gender       age  hypertension  heart_disease  ever_married  work_type  \
0       1  1.051434     -0.328602       4.185032             1          2   
1       0  0.786070     -0.328602      -0.238947             1          3   
2       1  1.626390     -0.328602       4.185032             1          2   
3       0  0.255342     -0.328602      -0.238947             1          2   
4       0  1.582163      3.043196      -0.238947             1          3   

   Residence_type  avg_glucose_level           bmi  smoking_status  
0               1           2.706375  1.001234e+00               1  
1               0           2.121559  4.615554e-16               2  
2               0          -0.005028  4.685773e-01               2  
3               1           1.437358  7.154182e-01               3  
4               0           1.501184 -6.357112e-01               2  


In [143]:
X.shape

(5110, 10)

In [144]:
X.dtypes

gender                 int64
age                  float64
hypertension         float64
heart_disease        float64
ever_married           int64
work_type              int64
Residence_type         int64
avg_glucose_level    float64
bmi                  float64
smoking_status         int64
dtype: object

In [145]:
X_train, X_test, y_train , y_test = train_test_split(X,y,test_size=0.2, random_state=42)
X_train.shape , X_test.shape

((4088, 10), (1022, 10))

In [115]:
# List of columns to one-hot encode
'''columns_to_encode = ['Residence_type', 'work_type', 'smoking_status','ever_married','gender']

# Iterate through each column and apply pd.get_dummies
for column in columns_to_encode:
    encoded_column = pd.get_dummies(df1[column], prefix=column)
    df1 = pd.concat([df1, encoded_column], axis=1)
    df1 = df1.drop(columns=[column],axis=1)

# Convert boolean to integers
df1 = df1.astype(int)'''

"columns_to_encode = ['Residence_type', 'work_type', 'smoking_status','ever_married','gender']\n\n# Iterate through each column and apply pd.get_dummies\nfor column in columns_to_encode:\n    encoded_column = pd.get_dummies(df1[column], prefix=column)\n    df1 = pd.concat([df1, encoded_column], axis=1)\n    df1 = df1.drop(columns=[column],axis=1)\n\n# Convert boolean to integers\ndf1 = df1.astype(int)"

In [146]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,accuracy_score

In [147]:
def evaluate_model(true,predicted):
    accuracy= accuracy_score(true,predicted)
    return accuracy

In [176]:
models = {
    "Decision Tree": DecisionTreeClassifier(),#criterion='entropy',splitter='best',max_depth=None
    "Random Forest Classifier": RandomForestClassifier( n_estimators=100,      # Reduce number of trees
    max_depth=10,          # Limit depth of trees
    min_samples_split=10,  # Require more samples to split
    min_samples_leaf=5,class_weight='balanced',    # Require more samples at leaf nodes
    random_state=42),#n_estimators=500, criterion='entropy', max_depth=None, random_state=42
    "XGBClassifier": xgb.XGBClassifier(),#objective='binary:logistic', max_depth=3, learning_rate=0.1, n_estimators=100
    "Logistic Regression": LogisticRegression()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_accuracy = evaluate_model(y_train, y_train_pred)

    model_test_accuracy = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Accuracy Score: {:.4f}".format(model_train_accuracy))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- R2 Score: {:.4f}".format(model_test_accuracy))
    r2_list.append(model_test_accuracy)
    
    print('='*35)
    print('\n')

Decision Tree
Model performance for Training set
- Accuracy Score: 1.0000
----------------------------------
Model performance for Test set
- R2 Score: 0.9139


Random Forest Classifier
Model performance for Training set
- Accuracy Score: 0.9369
----------------------------------
Model performance for Test set
- R2 Score: 0.8855


XGBClassifier
Model performance for Training set
- Accuracy Score: 0.9990
----------------------------------
Model performance for Test set
- R2 Score: 0.9393


Logistic Regression
Model performance for Training set
- Accuracy Score: 0.9543
----------------------------------
Model performance for Test set
- R2 Score: 0.9393




In [177]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'Accuracy_Score']).sort_values(by=["Accuracy_Score"],ascending=False)

,Model Name,Accuracy_Score
3,Logistic Regression,0.939335
2,XGBClassifier,0.939335
0,Decision Tree,0.913894
1,Random Forest Classifier,0.885519


In [178]:
rf_clf = RandomForestClassifier( n_estimators=100,      # Reduce number of trees
    max_depth=10,          # Limit depth of trees
    min_samples_split=10,  # Require more samples to split
    min_samples_leaf=5,
    class_weight='balanced',  # Adjust class weights    # Require more samples at leaf nodes
    random_state=42)
# Train the model
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)
print(f'Accuracy {accuracy_score(y_test,y_pred)}')
   

Accuracy 0.8855185909980431


In [179]:
print(y_train.value_counts())

stroke
0    3901
1     187
Name: count, dtype: int64


In [175]:
'''# Sample data for prediction
sample_data = {
    'gender': ['Female'],
    'age': [99],
    'hypertension': [1],
    'heart_disease': [1],
    'ever_married': ['Yes'],
    'work_type': ['Private'],
    'Residence_type': ['Urban'],
    'avg_glucose_level': [290],
    'bmi': [55],
    'smoking_status': ['smokes']
}

import pandas as pd
# Convert sample data to DataFrame
sample_df = pd.DataFrame(sample_data)

# Apply the same preprocessing as before
sample_df[cat_features] = sample_df[cat_features].apply(LabelEncoder().fit_transform)
sample_df[num_features] = StandardScaler().fit_transform(sample_df[num_features])

# Predict using the trained RandomForest model
#sample_prediction = rf_clf.predict(sample_df)
y_proba = rf_clf.predict_proba(X_test)[:,1]
y_pred = (y_proba > 0.3).astype(int)  # Adjust the threshold from 0.5 to 0.3, for example
# Print the prediction
print(f'Prediction: {"Stroke" if y_pred[0] == 1 else "No Stroke"}')'''

Prediction: No Stroke


In [180]:
import joblib
joblib.dump(rf_clf,"strokemodel.pkl")

['strokemodel.pkl']

In [ ]:
#df1.rename(columns={'work_type_Self-employed':'work_type_Selfemployed',
#           'smoking_status_formerly smoked':'smoking_status_formerly_smoked',
#           'smoking_status_never smoked':'smoking_status_never_smoked'},inplace=True)
#df1.columns

In [ ]:
#from sklearn.tree import DecisionTreeClassifier
#clf = DecisionTreeClassifier(criterion='entropy',splitter='best',max_depth=None)
#clf.fit(X_train,y_train)

In [ ]:
#from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,accuracy_score
#y_pred = clf.predict(X_test)
#print(f'Accuracy {accuracy_score(y_test,y_pred)}')

In [ ]:
'''from sklearn.ensemble import RandomForestClassifier

# Instantiate RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=None, random_state=42)

# Train the model
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)
print(f'Accuracy {accuracy_score(y_test,y_pred)}')'''

"from sklearn.ensemble import RandomForestClassifier\n\n# Instantiate RandomForestClassifier\nrf_clf = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=None, random_state=42)\n\n# Train the model\nrf_clf.fit(X_train, y_train)\ny_pred = rf_clf.predict(X_test)\nprint(f'Accuracy {accuracy_score(y_test,y_pred)}')"

In [ ]:
'''from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Example data preparation (replace with your actual data)

# Instantiate Logistic Regression model
logreg = LogisticRegression()

# Train the model
logreg.fit(X_train, y_train)

# Predict on validation set
y_pred = logreg.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')'''

Accuracy: 0.9393346379647749
